In [9]:
# # Server.ipynb
# Author: Diana Southard
# ECEN 5673: HW 1
#
# Write a simple UDP client/server program where the client sends a UDP message to ther server
# and the server sends a reply to the client.
#
# Based off the concepts shown on the Intro Python Networking - UDP YouTube video by largestpixel
# https://youtu.be/vNVMlXLGrTE
#
# NOTE: IP addresses must be changed to desired machines to communicate properly

# Import necessary libraries
import socket    # Necessary to establish connections
import datetime # Necessary to measure local clock times for measuring message transit times

# Global Variables
HOST = ""

# Can select any port, so long as it's used by both the client and the server
# Prefer to select ports over 1024 -> lower ports have default services assigned
PORT = 5454

# Message to be sent
data = 'Hello World'

# Python Function to setup for chosen scenario
def scenarioSetup(scenario, server):
    # Establish address of the client
    if scenario == 0:
        # "127.0.0.1" = client is on the same machine
        return "127.0.0.1"
    elif server == 1:
        # This machine is serving as the server, use its IP address
        return "192.168.0.6"
    else:
        # The other machine is the server, use its IP address
        return "192.168.0.5"

# Get user input for desired scenario
try:
    scenarioInput = int(input("Specify the desired scenario to run: 0 = Same Machine, 1 = Same LAN, 2 = Same WLAN, 3 = Different geographical locations: ")) 
    print("You have selected ", scenarioInput)
    if scenarioInput < 0 or scenarioInput > 4:
        print("That's not a correct input, defaulting to 0")
        scenarioInput = 0
except ValueError:
    print("That's not a correct input, defaulting to 0")
    scenarioInput = 0

try:
    serverSelect = int(input("Specify the desired worker to set up: 0 = Client, 1 = Server: "))
    print("You have selected ", serverSelect)
    if serverSelect < 0 or serverSelect > 1:
        print("That's not a correct input, defaulting to 0")
        serverSelect = 0
except ValueError:
    print("That's not a correct input, defaulting to 0")
    serverSelect = 0

# Setup lists to record message times
timeReceivedRecord = []
timeSentRecord = []

#Call scenario setup function
HOST = scenarioSetup(scenarioInput, serverSelect)

# Create a socket using IPv4 (AF_INET), with UDP protocol (SOCK_DGRAM) 
mySocket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

if (serverSelect):
    # Bind the server to the desired HOST and PORT
    try:
        mySocket.bind((HOST,PORT))
    except socket.error as e:
        print(str(e))

    print("Bound server to address " + HOST + "and port " + str(PORT))

# Send data by selecting socket, giving it the desired data, the address of the server/client, and which port to use
# Repeat 5 times
try:
    for i in range (0,5):
        # Work as server
        if (serverSelect):
            # Receive message from client
            print("Waiting to receive data from client")
            (clientData, clientAddress) = mySocket.recvfrom(1024) # Accept connections from outside, buffer size is 1024 bytes
            timeReceived = datetime.datetime.now().time() # Record time message was received
            timeReceived = timeReceived.isoformat() # Change to ISO format
            print('Server received message ', i, 'at time ', timeReceived)
            
            if clientData.decode('utf-8') != data:
                print("Incorrect client message: ", clientData)
                
            # Send reply to client
            mySocket.sendto(clientData, clientAddress) # Encode message to bytes
            timeSent = datetime.datetime.now().time()    # Record time message was sent
            timeSent = timeSent.isoformat() # Change to ISO format
            print('Server sent reply ', i, 'at time ', timeSent)
        
        # Work as client
        else:
            # Send message to server
            mySocket.sendto(data.encode('utf-8'), (HOST, PORT)) # Encode message to bytes
            timeSent = datetime.datetime.now().time()    # Record time message was sent
            timeSent = timeSent.isoformat() # Change to ISO format
            print('Client sent message ', i, 'at time ', timeSent)
            
            # Receive reply from server
            # recv: blocking call, wait for data from server
            print("Waiting to receive reply from server")
            serverData = mySocket.recv(1024) # buffer size is 1024 bytes, received only daya
            timeReceived = datetime.datetime.now().time() # Record time message was received
            timeReceived = timeReceived.isoformat() # Change to ISO format
            print('Client received reply ', i, 'at time ', timeReceived)
            
            if serverData.decode('utf-8') != data:
                print("Incorrect server reply received: ", serverData)
        
         # Append times to recorded times
        timeReceivedRecord.append(timeReceived)
        timeSentRecord.append(timeSent)

# Once all messages have been received/replied to, close the connection
# Perform this action even in the event of an exception in the try block
finally:
    mySocket.close()
    print("Socket closed")


# Print all times/results to outputfile
outputFileName = "UDPServerClientOutput" + str(scenarioInput) + str(serverSelect) + ".txt"
print('Writing times to ' + outputFileName)
with open(outputFileName, "w") as resultFile:
    resultFile.write('Host = ' + HOST + '\n')
    for time in timeReceivedRecord:
        resultFile.write('Received: ' + str(time) + '\n')
    for time in timeSentRecord:
        resultFile.write('Sent: ' + str(time) + '\n')
            
print('Finished')

Specify the desired scenario to run: 0 = Same Machine, 1 = Same LAN, 2 = Same WLAN, 3 = Different geographical locations: 1
You have selected  1
Specify the desired worker to set up: 0 = Client, 1 = Server: 0
You have selected  0
Client sent message  0 at time  21:36:40.276642
Waiting to receive reply from server
Client received reply  0 at time  21:36:40.286418
Client sent message  1 at time  21:36:40.286664
Waiting to receive reply from server
Client received reply  1 at time  21:36:40.287829
Client sent message  2 at time  21:36:40.288055
Waiting to receive reply from server
Client received reply  2 at time  21:36:40.289751
Client sent message  3 at time  21:36:40.289975
Waiting to receive reply from server
Client received reply  3 at time  21:36:40.304132
Client sent message  4 at time  21:36:40.304390
Waiting to receive reply from server
Client received reply  4 at time  21:36:40.308179
Socket closed
Writing times to UDPServerClientOutput10.txt
Finished


# 